# W261 Final Project - Factorization Machine Example

## Notebook Set-Up

In [18]:
# imports
import numpy as np
import pandas as pd

## Pre-Processing

In [5]:
toyDataRaw = ['1\t0\t5\t\t1\t26\tcat\tblue\t\tpizza',
            '0\t1\t10\t1\t\t12\tdog\tyellow\t\t',
            '0\t0\t\t0.5\t2\t45\tdog\t\tcar\tsteak']

In [46]:
# parse out label and features
toyDataParsed = []
for row in toyDataRaw:
    splitRow = row.split('\t')
    toyDataParsed.append((splitRow[0], splitRow[1:]))
toyDataParsed

[('1', ['0', '5', '', '1', '26', 'cat', 'blue', '', 'pizza']),
 ('0', ['1', '10', '1', '', '12', 'dog', 'yellow', '', '']),
 ('0', ['0', '', '0.5', '2', '45', 'dog', '', 'car', 'steak'])]

### Summarize Data

In [47]:
ncol = len(toyDataParsed[0][1])
nrow = len(toyDataParsed)
print(f'This toy exmaple  contains {nrow} rows and {ncol} columns, plus a label in index 0.')

This toy exmaple  contains 3 rows and 9 columns, plus a label in index 0.


In [48]:
def avgFeatures(row):
    count = 0
    feats = row[1][:]
    for feat in feats:
        if feat != '':
            count += 1
    return count

nonSparse = [avgFeatures(row) for row in toyDataParsed]

print("There is an average of", str(round(np.mean(nonSparse),2)), "populated features per observation.")

There is an average of 6.67 populated features per observation.


## One-Hot Encode Features

In [120]:
# binarize
def makeString(data):
    """Get list of features and make them into distinct strings according to column index"""
     #include label for SGD
    newData = []
    for r, row in enumerate(data):
        label = row[0]
        id_feats = []
        for i, value in enumerate(row[1], 1):
            if value=='':
                add='NA'
            else:
                add=value
            id_feats.append("v"+str(i)+"="+add)
        newData.append((label, id_feats))
    
    return newData
    
stringData = makeString(toyDataParsed)
print("Example of string-indexed features:")
stringData[0]

Example of string-indexed features:


('1',
 ['v1=0',
  'v2=5',
  'v3=NA',
  'v4=1',
  'v5=26',
  'v6=cat',
  'v7=blue',
  'v8=NA',
  'v9=pizza'])

In [122]:
def oneHotEncode(data):
    """turn indexed-string features into one-hot encoded features"""

    setFeats = set()
    for row in data:
        setFeats.update(row[1])
    listFeats = list(setFeats)
    print("Features:")
    print(listFeats)
    newData = np.zeros(shape=(len(data), len(listFeats)+1))

    for r, row in enumerate(data):
        newData[r][0] = row[0]    #first index is the label
        for var in row[1]:
            newData[r][listFeats.index(var)+1] = 1
            
    return newData, len(listFeats)
    
oneHotData, numFeats = oneHotEncode(stringData)
print("\nOne-hot encoded featres (first element is label):")
oneHotData[0]

Features:
['v8=car', 'v9=NA', 'v9=pizza', 'v3=NA', 'v5=12', 'v7=blue', 'v8=NA', 'v9=steak', 'v6=cat', 'v2=5', 'v1=0', 'v5=45', 'v4=2', 'v1=1', 'v6=dog', 'v4=NA', 'v7=NA', 'v2=10', 'v3=0.5', 'v3=1', 'v4=1', 'v5=26', 'v2=NA', 'v7=yellow']

One-hot encoded featres (first element is label):


array([1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 0.])

## Modeling using Stochastic Gradient Descent

In [106]:
# initialize model
b = 0.0
w_vector = np.random.normal(0.0, 0.02, (1, numFeats))
k = 2    #number of latent factors
V_matrix = np.random.normal(0.0, 0.02, (k, numFeats))   #k factors

print("Initialized weight vector W:")
w_vector

Initialized weight vector W:


array([[-0.00365852, -0.01959829,  0.03869003, -0.01711848, -0.00368362,
        -0.01940162,  0.00479304,  0.01115492, -0.01593766, -0.03149265,
        -0.02717584,  0.02590625, -0.01217509,  0.00166716,  0.0095978 ,
        -0.01828947,  0.02176057,  0.01835133, -0.00233219,  0.01166734,
         0.03017772,  0.00852512,  0.00766845, -0.01143573]])

In [123]:
def estimateGradient(record, k, b, w, V):
    """
        Compute the predicted probability AND return the gradients
        Args:
            record - label followed by binary feature values
        Model:
            b - bias term (scalar)
            w - linear weight vector (array)
            k - number of factors (def=2)
            V - factor matrix of size (d dimensions, k=2 factors)
        Returns:
            pair - ([label, predicted probability], [set of weight vectors in csr_matrix format])
    """
    
    label = record[0]
    feats = record[1:]
    
    # calculate P-hat    
    # start with linear weight dot product (X dot W)
    linear_sum = np.dot(w, feats)

    # factor matrix interaction sum
    factor_sum = 0.0
    lh_factor = [0.0]*k
    rh_factor = [0.0]*k
    for f in range(0, k):
        lh_factor[f] = np.dot(V[f][:], feats)  #KEY--this is used in v_grad matrix below
        rh_factor[f] = np.dot(V[f][:]**2, feats**2)
        factor_sum += (lh_factor[f]**2 - rh_factor[f])
    factor_sum = 0.5 * factor_sum
    
    y_hat = b + linear_sum + factor_sum
    
    p_hat = 1.0 / (1 + float(np.exp(-y_hat)))  #logit transformation
    
    #compute Gradients
    b_grad = p_hat - label    #the partial derivative of log-loss function wrt constant beta
    
    w_grad = b_grad*feats
    
    v_data = np.array([])
    for f in range(0, k):
        v_data = np.append(v_data, b_grad*(lh_factor[f]*feats - np.multiply(V[f][:], feats**2)))
    v_grad = np.reshape(v_data, newshape=(k, V.shape[1]))
    
    return ([label, p_hat], [b_grad, w_grad, v_grad])

In [124]:
# for one example
gradient = estimateGradient(oneHotData[0], k, b, w_vector, V_matrix)
print("(Label, predicted probability), [beta, w vector, V matrix]:")
gradient

(Label, predicted probability), [beta, w vector, V matrix]:


([1.0, 0.492026814119834],
 [-0.5079731858801659,
  array([-0.        , -0.        , -0.50797319, -0.50797319, -0.        ,
         -0.50797319, -0.50797319, -0.        , -0.50797319, -0.50797319,
         -0.50797319, -0.        , -0.        , -0.        , -0.        ,
         -0.        , -0.        , -0.        , -0.        , -0.        ,
         -0.50797319, -0.50797319, -0.        , -0.        ]),
  array([[-0.        , -0.        , -0.00130069,  0.00859022, -0.        ,
           0.01148334,  0.00485216, -0.        ,  0.00567576,  0.00050664,
          -0.00242005, -0.        ,  0.        ,  0.        , -0.        ,
          -0.        ,  0.        ,  0.        , -0.        ,  0.        ,
           0.00717801, -0.00059487, -0.        , -0.        ],
         [-0.        ,  0.        , -0.02101515, -0.0006494 ,  0.        ,
           0.01148509,  0.02197304, -0.        ,  0.00701873,  0.013716  ,
           0.0151001 , -0.        , -0.        , -0.        ,  0.        ,
   

In [125]:
def logLoss(pair):
    """parallelize log loss
        input: ([label, prob], [b_grad, w_grad, v_grad])
    """
    y = pair[0][1]
    
    eps = 1.0e-16
    if pair[0][1] == 0:
        p_hat = eps
    elif pair[0][1] == 1:
        p_hat = 1-eps
    else:
        p_hat = pair[0][1]
    
    return float(-(y * np.log(p_hat) + (1-y) * np.log(1-p_hat)))

In [126]:
logLoss(gradient)

0.6930200317847577

In [127]:
# update weights
learningRate = 0.1

wGrad_reduce = np.zeros((1, numFeats))
for r in range(0, nrow):
    gradient = estimateGradient(oneHotData[r], k, b, w_vector, V_matrix)
    wGrad_reduce += gradient[1][1]
w_update = wGrad_reduce / nrow

w_new = w_vector - learningRate*w_update

print("New weight vector W")
w_new

New weight vector W


array([[-0.02058979, -0.03616224,  0.05562247, -0.00018604, -0.02024757,
        -0.00246918,  0.00516153, -0.00577635,  0.00099478, -0.01456022,
        -0.02717468,  0.00897498, -0.02910637, -0.01489679, -0.02389743,
        -0.03485342,  0.00482929,  0.00178738, -0.01926347, -0.00489661,
         0.04711016,  0.02545756, -0.00926282, -0.02799968]])

In [130]:
# update V matrix

vGrad_reduce = np.zeros((k, numFeats))
for r in range(0, nrow):
    gradient = estimateGradient(oneHotData[r], k, b, w_vector, V_matrix)
    vGrad_reduce += gradient[1][2]
v_update = vGrad_reduce / nrow

V_new = V_matrix - learningRate*v_update

print("New factor matrix V weights:")
V_new

New factor matrix V weights:


array([[-0.01368164, -0.01584299, -0.01087652,  0.0082651 , -0.02212212,
         0.01386408,  0.00127107,  0.00037909,  0.00262483, -0.00737883,
        -0.0122236 , -0.02944858,  0.02788355,  0.02471118, -0.01435605,
        -0.03359133,  0.0092739 ,  0.0508294 , -0.020057  ,  0.02658089,
         0.00553209, -0.00951056, -0.00089464, -0.02839795],
       [-0.01690218,  0.01003133, -0.05893364, -0.01952033,  0.01327944,
         0.00396325,  0.02487061, -0.00645672, -0.00468038,  0.00828068,
         0.01025487, -0.0030888 , -0.00948079, -0.0149556 ,  0.01553272,
        -0.03569664,  0.0139793 , -0.00944846,  0.02004775,  0.02208024,
        -0.00989921,  0.02483253,  0.0219458 , -0.03734097]])